# Exemplo de Spark Streaming con sockets

1. Creamos a spark-session:

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession.builder \
    .appName("StructuredWordCount") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

print("Versión de spark: ",spark.version)


2. Creamos un dataframe a partir dunha *orixe* de datos. Neste caso imos empregar *socket*, que non se recomenda en produción:
> Para que este exemplo funcione temos que lanzar o netcat dende un terminal:
```bash
nc -lk 9999
```


In [ ]:
lines = spark \
         .readStream \
         .format("socket") \
         .option("host", "localhost") \
         .option("port",9999) \
         .load()

3. Xeramos un novo dataframe que conta as ocorrencias de cada palabra a partir da anterior.

In [ ]:
from pyspark.sql.functions import regexp_replace
# Dividimos a liña en palabras
words = lines.select(
        explode(
split(lines.value, " ")
   ).alias("word")
)
# Explicación
# 1. split converte unha liña nunha lista de palabras
# 2. explode converte cada palabra da lista a unha fila do dataframe

 # Xeramos o wordcount
wordCounts = words.groupBy("word").count()

4. Iniciamos a consulta indicando o destino (*sink*)

In [ ]:
query = (
    wordCounts.writeStream
        # modo actualización
        .outputMode("update")
        # formato memoria: útil para probas en notebooks
        .format("memory")
        # nome da vista para poder acceder aos resultados
        .queryName("consulta1")
        # o proceso dispárase cada 30 segundos
        .trigger(processingTime="30 seconds")
        # iníciase a consulta de streaming
        .start()
)


5. Mostramos

In [ ]:
from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT * FROM consulta1').show())
    sleep(1)